In [ ]:
import nltk
from nltk.collocations import  BigramCollocationFinder
from nltk.metrics import  BigramAssocMeasures
from nltk.probability import  FreqDist,ConditionalFreqDist
import pandas as pd
import jieba
import re

In [2]:
def bag_of_words(words):
    return dict([(word, True) for word in words])

def bigram(words, score_fn=BigramAssocMeasures.chi_sq, n=1000):
    bigram_finder = BigramCollocationFinder.from_words(words)  #把文本变成双词搭配的形式
    bigrams = bigram_finder.nbest(score_fn, n) #使用了卡方统计的方法，选择排名前1000的双词
    return bag_of_words(bigrams)

def bigram_words(words, score_fn=BigramAssocMeasures.chi_sq, n=1000):
    bigram_finder = BigramCollocationFinder.from_words(words)
    bigrams = bigram_finder.nbest(score_fn, n)
    return bag_of_words(words + bigrams)  #所有词和（信息量大的）双词搭配一起作为特征

In [3]:
MD=pd.read_csv("C:\\Users\\Adam\\Desktop\\output\\freedom.csv",encoding="ANSI")  #讀取檔案 CSV須加編碼



business = MD[MD["版別"] == "business"]['文章內容']
world = MD[MD["版別"] == "world"]['文章內容']
politics = MD[MD["版別"] == "politics"]['文章內容']


jieba.set_dictionary('C:\\Users\\Adam\\Desktop\\help\\dict.txt.big.txt')   #設定結巴的繁體字典
jieba.load_userdict("C:\\Users\\Adam\\Desktop\\help\\userdict.txt")  #使用者補充字典

bus=[]   #目標形式是[['','',''],['','',''],['','','']...]

for article in business:
    bus_2=[]
    newcontent=re.sub("〔.*?〕|（.*?）",'',str(article))       #*?獲取最短滿足條件
    newcontent_2=re.sub("\d|[\s+\.\!\/_,$%^*(+\"\'《》「」]+|[+-—！，。？、~@#￥%……&*()（）:]+",'',newcontent)
    words = jieba.cut(newcontent_2,cut_all=False)
    for w in words:
        bus_2.append(w)
    bus.append(bus_2)

wor=[]

for article in world:
    wor_2=[]
    newcontent=re.sub("〔.*?〕|（.*?）",'',str(article))       #*?獲取最短滿足條件
    newcontent_2=re.sub("\d|[\s+\.\!\/_,$%^*(+\"\'《》「」]+|[+-—！，。？、~@#￥%……&*()（）:]+",'',newcontent)
    words = jieba.cut(newcontent_2,cut_all=False)
    for w in words:
        wor_2.append(w)
    wor.append(wor_2)
    
pol=[]

for article in politics:
    pol_2=[]
    newcontent=re.sub("〔.*?〕|（.*?）",'',str(article))       #*?獲取最短滿足條件
    newcontent_2=re.sub("\d|[\s+\.\!\/_,$%^*(+\"\'《》「」]+|[+-—！，。？、~@#￥%……&*()（）:]+",'',newcontent)
    words = jieba.cut(newcontent_2,cut_all=False)
    for w in words:
        pol_2.append(w)    
    pol.append(pol_2)
    


Building prefix dict from C:\Users\Adam\Desktop\help\dict.txt.big.txt ...
Loading model from cache C:\Users\Adam\AppData\Local\Temp\jieba.u2a5ff907f0989b23cd0cb4eed1d5c332.cache
Loading model cost 1.325 seconds.
Prefix dict has been built succesfully.


In [4]:
word_fd = FreqDist() #可统计所有词的词频
cond_word_fd = ConditionalFreqDist() #可统计积极文本中的词频和消极文本中的词频

for word2 in bus:
    for word in word2:
        word_fd[word] += 1
        cond_word_fd['bus'][word] += 1

for word2 in wor:
    for word in word2:
        word_fd[word] += 1
        cond_word_fd['wor'][word] += 1

for word2 in pol:
    for word in word2:
        word_fd[word] += 1
        cond_word_fd['pol'][word] += 1

bus_word_count = cond_word_fd['bus'].N() #財經新聞的数量
wor_word_count = cond_word_fd['wor'].N() #國際新聞的数量
pol_word_count = cond_word_fd['pol'].N() #國際新聞的数量

total_word_count = bus_word_count + wor_word_count + pol_word_count



In [5]:
word_scores={}

for word, freq in word_fd.items():
    bus_score = BigramAssocMeasures.chi_sq(cond_word_fd['bus'][word],  (freq, bus_word_count), total_word_count) #计算积极词的卡方统计量，这里也可以计算互信息等其它统计量
    wor_score = BigramAssocMeasures.chi_sq(cond_word_fd['wor'][word],  (freq, wor_word_count), total_word_count)
    pol_score = BigramAssocMeasures.chi_sq(cond_word_fd['pol'][word],  (freq, pol_word_count), total_word_count)
    word_scores[word] = bus_score + wor_score + pol_score #一个词的信息量等于积极卡方统计量加上消极卡方统计量

In [6]:
best_vals = sorted(word_scores.items(), key=lambda item:item[1],  reverse=True)[:300] #把词按信息量倒序排序。number是特征的维度，是可以不断调整直至最优的
best_words = set([w for w,s in best_vals])
ss=dict([(word, True) for word in best_words])

In [7]:
busfeature = []
for items in bus:
    a = {}
    for item in items:
        if item in ss.keys():
            a[item]='True'
    busWords = [a,'bus'] #为积极文本赋予"pos"
    busfeature.append(busWords)

worfeature = []
for items in wor:
    a = {}
    for item in items:
        if item in ss.keys():
            a[item]='True'
    worWords = [a,'wor'] #为积极文本赋予"pos"
    worfeature.append(worWords)
    
polfeature = []
for items in pol:
    a = {}
    for item in items:
        if item in ss.keys():
            a[item]='True'
    polWords = [a,'pol'] #为积极文本赋予"pos"
    polfeature.append(polWords)

In [8]:
polfeature

[[{'上': 'True',
   '人': 'True',
   '他': 'True',
   '以來': 'True',
   '副': 'True',
   '參選': 'True',
   '受訪': 'True',
   '在': 'True',
   '大家': 'True',
   '她': 'True',
   '市民': 'True',
   '市長': 'True',
   '年': 'True',
   '新': 'True',
   '新北': 'True',
   '景峻': 'True',
   '民調': 'True',
   '民進黨': 'True',
   '沒有': 'True',
   '的': 'True',
   '立委': 'True',
   '總統': 'True',
   '英文': 'True',
   '蔡': 'True',
   '行政院長': 'True',
   '被': 'True',
   '要': 'True',
   '說': 'True',
   '選舉': 'True',
   '陳': 'True',
   '黨': 'True',
   '黨中央': 'True',
   '；': 'True'},
  'pol'],
 [{'上': 'True',
   '之': 'True',
   '受訪': 'True',
   '召開': 'True',
   '和': 'True',
   '在': 'True',
   '對於': 'True',
   '市議員': 'True',
   '提名': 'True',
   '據': 'True',
   '月': 'True',
   '民進黨': 'True',
   '決議': 'True',
   '沒有': 'True',
   '的': 'True',
   '總統': 'True',
   '英文': 'True',
   '蔡': 'True',
   '被': 'True',
   '議員': 'True',
   '議長': 'True',
   '選區': 'True',
   '選舉': 'True',
   '陳': 'True',
   '黨': 'True',
   '黨內': 'True',
   '黨部'

In [9]:
#從這裡修改

from random import shuffle
 #把积极文本的排列随机化
shuffle(busfeature)
shuffle(polfeature)
shuffle(worfeature)
size = int(len(polfeature)-12)


train =  busfeature[75:]+polfeature[75:]+worfeature[75:]#训练集(70%)
test = busfeature[:75]+polfeature[:75]+worfeature[:75]#验证集(30%)

In [10]:
data,tag = zip(*test) #分离测试集合的数据和标签，便于验证和测试

In [18]:
train

[[{'一度': 'True',
   '不': 'True',
   '主要': 'True',
   '來到': 'True',
   '元': 'True',
   '及': 'True',
   '台北': 'True',
   '台股': 'True',
   '在': 'True',
   '市場': 'True',
   '帶動': 'True',
   '指數': 'True',
   '日本': 'True',
   '月': 'True',
   '期貨': 'True',
   '漲幅': 'True',
   '的': 'True',
   '約': 'True',
   '美股': 'True',
   '股': 'True',
   '股市': 'True',
   '賣壓': 'True',
   '跌': 'True',
   '跌幅': 'True',
   '重挫': 'True',
   '開盤': 'True',
   '震盪': 'True',
   '點': 'True',
   '；': 'True'},
  'bus'],
 [{'他': 'True',
   '副總裁': 'True',
   '匯率': 'True',
   '可望': 'True',
   '台北': 'True',
   '台灣': 'True',
   '大家': 'True',
   '央行': 'True',
   '對於': 'True',
   '彭': 'True',
   '持續': 'True',
   '新台幣': 'True',
   '楊金龍': 'True',
   '淮南': 'True',
   '的': 'True',
   '總裁': 'True',
   '行政院': 'True',
   '說': 'True',
   '超過': 'True',
   '銀行': 'True'},
  'bus'],
 [{'去年': 'True',
   '及': 'True',
   '市場': 'True',
   '成長': 'True',
   '數位': 'True',
   '新': 'True',
   '智慧': 'True',
   '的': 'True',
   '經濟': 'True',
   '董事

In [11]:
data

({'一度': 'True',
  '上漲': 'True',
  '下跌': 'True',
  '今年': 'True',
  '利率': 'True',
  '升息': 'True',
  '在': 'True',
  '基金': 'True',
  '報點': 'True',
  '工業': 'True',
  '市場': 'True',
  '帶動': 'True',
  '年': 'True',
  '指數': 'True',
  '收': 'True',
  '收盤': 'True',
  '月': 'True',
  '漲幅': 'True',
  '的': 'True',
  '盤中': 'True',
  '美股': 'True',
  '聯準': 'True',
  '股價': 'True',
  '跌幅': 'True',
  '道瓊': 'True',
  '預期': 'True',
  '預計': 'True'},
 {'上': 'True',
  '今年': 'True',
  '他': 'True',
  '全球': 'True',
  '台灣': 'True',
  '和': 'True',
  '在': 'True',
  '報導': 'True',
  '年': 'True',
  '指數': 'True',
  '的': 'True',
  '組織': 'True',
  '綜合': 'True',
  '美國': 'True',
  '金融': 'True',
  '金額': 'True',
  '香港': 'True'},
 {'上漲': 'True',
  '主要': 'True',
  '價格': 'True',
  '億元': 'True',
  '原油': 'True',
  '去年': 'True',
  '及': 'True',
  '可望': 'True',
  '客戶': 'True',
  '市場': 'True',
  '帶動': 'True',
  '年': 'True',
  '成長': 'True',
  '月': 'True',
  '營收': 'True',
  '營運': 'True',
  '產品': 'True',
  '總經理': 'True',
  '美國': 'True',
  '

In [12]:
def score(classifier):
    classifier = SklearnClassifier(classifier) 
    classifier.train(train) #训练分类器
    pred = classifier.classify_many(data) #给出预测的标签
    n = 0
    s = len(pred)
    for i in range(0,s):
        if pred[i]==tag[i]:
            n = n+1
    return n/s #分类器准确度

In [13]:
import sklearn
from nltk.classify.scikitlearn import  SklearnClassifier
from sklearn.svm import SVC, LinearSVC,  NuSVC
from sklearn.naive_bayes import  MultinomialNB, BernoulliNB
from sklearn.linear_model import  LogisticRegression
from sklearn.metrics import  accuracy_score

In [14]:
print('BernoulliNB`s accuracy is %f'  %score(BernoulliNB()))
print('MultinomiaNB`s accuracy is %f'  %score(MultinomialNB()))
print('LogisticRegression`s accuracy is  %f' %score(LogisticRegression()))
#print('SVC`s accuracy is %f'  %score(SVC()))
#print('LinearSVC`s accuracy is %f'  %score(LinearSVC()))
#print('NuSVC`s accuracy is %f'  %score(NuSVC()))

BernoulliNB`s accuracy is 0.888889
MultinomiaNB`s accuracy is 0.888889
LogisticRegression`s accuracy is  0.924444


In [15]:

classifier = SklearnClassifier(BernoulliNB()) 
ss=classifier.train(train) #训练分类器
pred = classifier.classify_many(data) #给出预测的标签
n = 0
s = len(pred)
for i in range(0,s):
    if pred[i]==tag[i]:
        n = n+1

In [16]:
pred

['bus',
 'wor',
 'bus',
 'wor',
 'wor',
 'bus',
 'bus',
 'bus',
 'bus',
 'bus',
 'bus',
 'bus',
 'bus',
 'bus',
 'bus',
 'bus',
 'bus',
 'bus',
 'bus',
 'bus',
 'bus',
 'bus',
 'bus',
 'bus',
 'bus',
 'wor',
 'wor',
 'pol',
 'bus',
 'bus',
 'pol',
 'bus',
 'bus',
 'bus',
 'bus',
 'bus',
 'bus',
 'bus',
 'bus',
 'pol',
 'bus',
 'bus',
 'bus',
 'bus',
 'wor',
 'wor',
 'wor',
 'bus',
 'bus',
 'wor',
 'bus',
 'bus',
 'bus',
 'bus',
 'bus',
 'bus',
 'pol',
 'bus',
 'bus',
 'bus',
 'bus',
 'bus',
 'bus',
 'bus',
 'bus',
 'bus',
 'bus',
 'bus',
 'wor',
 'bus',
 'bus',
 'bus',
 'bus',
 'bus',
 'bus',
 'pol',
 'pol',
 'wor',
 'pol',
 'pol',
 'bus',
 'pol',
 'wor',
 'pol',
 'wor',
 'pol',
 'pol',
 'pol',
 'pol',
 'pol',
 'pol',
 'pol',
 'pol',
 'pol',
 'pol',
 'pol',
 'pol',
 'pol',
 'pol',
 'pol',
 'pol',
 'pol',
 'wor',
 'pol',
 'pol',
 'pol',
 'wor',
 'pol',
 'pol',
 'pol',
 'pol',
 'pol',
 'wor',
 'pol',
 'pol',
 'pol',
 'pol',
 'pol',
 'pol',
 'pol',
 'pol',
 'pol',
 'pol',
 'pol',
 'pol',


In [19]:
pred = classifier.classify_many({'今年': 'True'})
pred

['bus']